In [1]:
from google.colab import files
files.upload()  # Run this and upload the kaggle.json file you just downloaded


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ashish755","key":"6e94e145ff14bd6d60c839b5d4b2b3ed"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [3]:
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d gti-upm/leapgestrecog

Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.12G/2.13G [00:22<00:00, 109MB/s]
100% 2.13G/2.13G [00:22<00:00, 102MB/s]


In [6]:
!unzip leapgestrecog.zip

Streaming output truncated to the last 5000 lines.
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0001.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0002.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0003.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0004.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0005.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0006.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0007.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0008.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0009.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0010.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0011.png  
  inflating: leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0012.png  
  inflating: leapgestrecog/leapGestRe

# Optional

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
print(os.listdir("leapgestrecog/leapGestRecog"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import IPython.display
path='leapgestrecog/leapGestRecog'
folders=os.listdir(path)
folders=set(folders)

import codecs
import json


different_classes=os.listdir(path+'/'+'00')
different_classes=set(different_classes)




print("The different classes that exist in this dataset are:")
print(different_classes,sep='\n')

In [ ]:
classes=dict()
names_of_classes=dict()
for i in different_classes:
    classes[int(i.split('_')[0])]='_'.join(i.split('_')[1:])
    names_of_classes['_'.join(i.split('_')[1:])]=int(i.split('_')[0])




print(classes)

In [ ]:
x=[]
z=[]
y=[]#converting the image to black and white
threshold=200
import cv2


for i in folders:
    print('***',i,'***')
    subject=path+'/'+i
    subdir=os.listdir(subject)
    subdir=set(subdir)
    for j in subdir:
        print(j)
        images=os.listdir(subject+'/'+j)
        for k in images:
            results=dict()
            results['y']=j.split('_')[0]
            img = cv2.imread(subject+'/'+j+'/'+k,0)
            img=cv2.resize(img,(int(160),int(60)))

            ret, imgf = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            imgD=np.asarray(img,dtype=np.float64)
            z.append(imgD)
            imgf=np.asarray(imgf,dtype=np.float64)
            x.append(imgf)
            y.append(int(j.split('_')[0]))
            results['x']=imgf

print(list(set(y)))

In [ ]:
#sample black and white image from each class
l = []
list_names = []
for i in range(10):
    l.append(0)
for i in range(len(x)):
    if(l[y[i] - 1] == 0):
        l[y[i] - 1] = i
        if(len(np.unique(l)) == 10):
            break
for i in range(len(l)):
    %matplotlib inline
    print("Class Label: " + str(i + 1))
    plt.imshow(np.asarray(z[l[i]]), cmap  =cm.gray)
    plt.show()
    plt.imshow(np.asarray(x[l[i]]), cmap = cm.gray)
    plt.show()

In [ ]:
x=np.array(x)
y=np.array(y)
y = y.reshape(len(x), 1)
print(x.shape)
print(y.shape)
print(max(y),min(y))

In [ ]:
np.unique(y)

In [ ]:
import keras
from keras.utils import to_categorical
def one_hot_encoded(y):
    p=list(np.unique(y))
    dictionary=dict()
    final_result=[]
    for i in range(len(p)):
        dictionary[p[i]]=i
    for i in y:
        actual=[0 for j in range(len(p))]
        actual[dictionary[i[0]]]=1
        final_result.append(actual)
    return np.array(final_result)



y_data = one_hot_encoded(y)
print(y_data[0])
print(type(y_data))

In [ ]:
x_data = x.reshape((len(x), 60, 160, 1))

x_data/=255
print(x_data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5)

In [ ]:
from keras import layers
from keras import models

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(60, 160,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate))

In [ ]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)
print("Accuracy:" + str(acc))

In [ ]:
model.save('model.h5')

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import gradio as gr

# Load your trained model
model = tf.keras.models.load_model('model.h5')

# Class labels
class_labels = {10: 'down', 4: 'fist_moved', 2: 'l', 7: 'ok', 3: 'fist', 5: 'thumb', 9: 'c', 8: 'palm_moved', 6: 'index', 1: 'palm'}

def predict_hand_gesture(image):
    # Resize and convert to grayscale as per training requirements
    image = image.convert("L").resize((160, 60))  # Resize to (160, 60) and convert to grayscale
    image = np.array(image, dtype=np.float64) / 255.0
    image = np.expand_dims(image, axis=(0, -1))  # Add batch and channel dimensions

    # Predict class
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)[0]
    return class_labels.get(predicted_class, "Unknown")

# Gradio Interface
interface = gr.Interface(
    fn=predict_hand_gesture,
    inputs=gr.Image(type="pil"),  # Capture image from webcam or upload
    outputs="text",
    title="Hand Gesture Recognition",
    description="Capture or upload an image of a hand gesture for class prediction."
)

# Launch the app
interface.launch()


# optional 2

In [8]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [9]:
# Import necessary libraries
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [10]:
# Prepare data
path = 'leapgestrecog/leapGestRecog'
folders = os.listdir(path)

In [11]:
# Classes dictionary
classes = {}
different_classes = os.listdir(path + '/00')
for i in different_classes:
    classes[int(i.split('_')[0])] = '_'.join(i.split('_')[1:])

In [12]:
# Load images and labels
x, y = [], []
for folder in folders:
    subject = path + '/' + folder
    for subdir in os.listdir(subject):
        label = int(subdir.split('_')[0])
        for img_name in os.listdir(subject + '/' + subdir):
            img = cv2.imread(subject + '/' + subdir + '/' + img_name, 0)
            img = cv2.resize(img, (160, 60))
            _, imgf = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            x.append(imgf)
            y.append(label)

In [14]:
# Convert lists to numpy arrays
x = np.array(x).reshape((len(x), 60, 160, 1)) / 255.0
y = np.array(y).reshape(len(y), 1)

# Adjust y to start from 0 instead of 1 (to fit to_categorical)
y -= 1

# One-hot encode labels
num_classes = len(np.unique(y))
y_data = tf.keras.utils.to_categorical(y, num_classes=num_classes)

In [15]:
# Split data into train, validation, and test sets
x_train, x_further, y_train, y_further = train_test_split(x, y_data, test_size=0.2)
x_validate, x_test, y_validate, y_test = train_test_split(x_further, y_further, test_size=0.5)

In [16]:
# Build model
model = models.Sequential([
    layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(60, 160, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Compile and train model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_validate, y_validate))

# Evaluate the model on the test set
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.0979 - loss: 2.3029 - val_accuracy: 0.0900 - val_loss: 2.3030
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0993 - loss: 2.3027 - val_accuracy: 0.0860 - val_loss: 2.3037
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1017 - loss: 2.3027 - val_accuracy: 0.0950 - val_loss: 2.3031
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1019 - loss: 2.3026 - val_accuracy: 0.0900 - val_loss: 2.3030
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1071 - loss: 2.3026 - val_accuracy: 0.0950 - val_loss: 2.3032
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1012 - loss: 2.3026 - val_accuracy: 0.0950 - val_loss: 2.3032
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0997 - loss: 2.3027 - val_accuracy: 0.0900 - val_loss: 2.3033
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1075 - loss: 2.3024 - val_accuracy: 0

In [18]:
# Generate confusion matrix and classification report
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

class_labels = {10: 'down', 4: 'fist_moved', 2: 'l', 7: 'ok', 3: 'fist', 5: 'thumb', 9: 'c', 8: 'palm_moved', 6: 'index', 1: 'palm'}
class_report = classification_report(y_true_classes, y_pred_classes, target_names=[class_labels[i] for i in sorted(class_labels)])
print("\nClassification Report:")
print(class_report)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0 190]
 [  0   0   0   0   0   0   0   0   0 192]
 [  0   0   0   0   0   0   0   0   0 211]
 [  0   0   0   0   0   0   0   0   0 232]
 [  0   0   0   0   0   0   0   0   0 216]
 [  0   0   0   0   0   0   0   0   0 187]
 [  0   0   0   0   0   0   0   0   0 182]
 [  0   0   0   0   0   0   0   0   0 194]
 [  0   0   0   0   0   0   0   0   0 195]
 [  0   0   0   0   0   0   0   0   0 201]]

Classification Report:
              precision    recall  f1-score   support

        palm       0.00      0.00      0.00       190
           l       0.00      0.00      0.00       192
        fist       0.00      0.00      0.00       211
  fist_moved       0.00      0.00      0.00       232
       thumb       0.00      0.00      0.00       216
       index       0.00      0.00      0.00       187
          ok       0.00      0.00      0.00       182
  palm_moved       0.00      0.00      0.00       194


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
